In [21]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
%pip install litellm
%pip install -qU trulens_eval pydantic fastapi kaleido python-multipart uvicorn cohere openai tiktoken "llama-index"
%pip install transformers
%pip install sentence-transformers
%pip install pinecone-client
%pip install datasets
%pip install accelerate
%pip install einops
%pip install langchain
%pip install bitsandbytes
%pip install matplotlib seaborn tqdm
%pip install chromadb
%pip install evaluate
%pip install rouge_score
%pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.5/662.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [4]:
!pip install dill

In [7]:
from torch import cuda, bfloat16
import os
import transformers
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
model_id = 'meta-llama/Llama-2-13b-chat-hf'

In [8]:
embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'
device = 'cuda:0' # make sure you are on gpu

embed_model = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
from transformers import LlamaTokenizer
hf_auth = os.environ.get('HF_AUTH')
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf",use_auth_token=hf_auth)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1917: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [10]:
bitsAndBites_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [11]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=os.environ.get('HF_AUTH')
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [12]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bitsAndBites_config,
    device_map='auto',
    token=os.environ.get('HF_AUTH')
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    # we pass model parameters here too
    temperature=0.01,
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [14]:
import os
from pinecone import Pinecone, ServerlessSpec

# Get Pinecone API key and environment from environment variables
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

# Create an instance of the Pinecone class
pc = Pinecone(api_key=pinecone_api_key)

In [15]:
index_name = 'inlpt-project'
index = pc.Index(index_name)

In [16]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to get spaCy embedding for a text string
def get_embedding_query(text):
    doc = nlp(text)
    # Use the vector for the entire document
    return doc.vector


In [17]:
from langchain.vectorstores import Pinecone
### your code ###
vectorstore = Pinecone(index, embed_model.embed_query, 'text')
### your code ###

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [18]:
query = "What is the name of the Cooperation Agreement between the European Atomic Energy Community and the International Atomic Energy Agency?"

In [19]:

### your code ###
vectorstore.similarity_search(
    query,  # the search query
    k=3  # returns top 3 most relevant chunks of text
)
### your code ###

[Document(page_content='the Agreement for cooperation between the European Atomic Energy Community represented by the Commission and the Department of Energy of the United States of America in the field of fusion energy research and development(2001/411/Euratom)THE COUNCIL OF THE EUROPEAN UNION,Having regard to the Treaty establishing the European Atomic Energy Community, and in particular Article 101, second paragraph thereof,Having regard to the proposal from the Commission,Whereas:(1) The Agreement for cooperation in the peaceful uses of nuclear energy between the European Atomic Energy Community and the United States of America,', metadata={'CELEX number': '32001D0411'}),
 Document(page_content='EUROPEAN ATOMIC ENERGY COMMUNITY AND THE INTERNATIONAL ATOMIC ENERGY AGENCY (75/780/Euratom)   Article I  Cooperation and consultation  The International Atomic Energy Agency (hereinafter referred to as "the Agency") and the European Atomic Energy Community (hereinafter referred to as "the 

In [20]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

In [21]:
from langchain.chains import RetrievalQA


rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    verbose=True,
    retriever=vectorstore.as_retriever(search_kwargs={"k":5}),
    chain_type_kwargs={
        "verbose": True },

)


In [22]:
llm(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Keyword arguments {'add_special_tokens': False} not recognized.


'\n\nA. Euratom-IAEA Agreement\nB. Joint Declaration on Nuclear Safety\nC. Cooperation in Nuclear Safety and Security\nD. Framework for Nuclear Cooperation\n\nAnswer: A. Euratom-IAEA Agreement.\n\nThe Cooperation Agreement between the European Atomic Energy Community (Euratom) and the International Atomic Energy Agency (IAEA) is called the Euratom-IAEA Agreement. This agreement aims to promote cooperation in the fields of nuclear energy, safety, and security, as well as to facilitate the exchange of information and expertise between the two organizations.'

In [23]:
rag_pipeline(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Keyword arguments {'add_special_tokens': False} not recognized.




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

the Agreement for cooperation between the European Atomic Energy Community represented by the Commission and the Department of Energy of the United States of America in the field of fusion energy research and development(2001/411/Euratom)THE COUNCIL OF THE EUROPEAN UNION,Having regard to the Treaty establishing the European Atomic Energy Community, and in particular Article 101, second paragraph thereof,Having regard to the proposal from the Commission,Whereas:(1) The Agreement for cooperation in the peaceful uses of nuclear energy between the European Atomic Energy Community and the United States of America,

EUROPEAN ATOMIC ENERGY COMMUNITY AND THE INTERNATIONAL ATOMIC ENERGY AGENC

{'query': 'What is the name of the Cooperation Agreement between the European Atomic Energy Community and the International Atomic Energy Agency?',
 'result': ' The name of the Cooperation Agreement between the European Atomic Energy Community and the International Atomic Energy Agency is "Cooperation Agreement between the European Atomic Energy Community and the International Atomic Energy Agency" (75/780/Euratom).'}

In [27]:
%pip install lmqg

In [28]:
from pprint import pprint
from lmqg import TransformersQG
import pandas as pd

In [29]:
model = TransformersQG(language='en', model='lmqg/t5-base-squad-qag')

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2850: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io

/usr/local/lib/python3.10/dist-packages


In [30]:
try:
    df = pd.read_csv("/chunks.csv", error_bad_lines=False)
except pd.errors.ParserError as e:
    print("Error parsing CSV file:", e)
    exit()

for index, row in df.head(2).iterrows():
    context = row['text']

    try:
        question_answer = model.generate_qa(context)[:20]
    except Exception as e:
        print("Error generating QA for row", index + 1, ":", e)
        continue

    print("Questions generated for row", index + 1)
    pprint(question_answer)
    print("----------------------------------")


<ipython-input-30-a88cf700d8c8>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/chunks.csv", error_bad_lines=False)
100%|██████████| 1/1 [00:00<00:00, 116.83it/s]


Questions generated for row 1
[('What is the name of the Cooperation Agreement between the European Atomic '
  'Energy Community and the International Atomic Energy Agency?',
  'Avis juridique important'),
 ('What is the name of the Swedish special edition of the Cooperation '
  'Agreement?',
  'Chapter 11 Volume 3 P. 0006'),
 ('What is the name of the Portuguese special edition of the Cooperation '
  'Agreement between the European Atomic Energy Community and the '
  'International Atomic Energy Agency?',
  'Chapter 12 Volume 2 P. 0092'),
 ('What is the name of the Cooperation Agreement between the European Atomic '
  'Energy Community and the International Atomic Energy Agency?',
  'Avis juridique important Cooperation Agreement')]
----------------------------------


100%|██████████| 1/1 [00:00<00:00, 384.59it/s]


Questions generated for row 2
[('What does the International Atomic Energy Agency stand for?', 'the Agency'),
 ('What does the European Atomic Energy Community stand for?', 'the Community'),
 ('What does the International Atomic Energy Agency stand for?', 'the Agency'),
 ('What does the European Atomic Energy Community stand for?', 'the Community')]
----------------------------------


In [ ]:
print(context)

Avis juridique important Cooperation Agreement between the European Atomic Energy Community and the International Atomic Energy Agency     Official Journal L 329 , 23/12/1975 P. 0028 - 0029 Finnish special edition: Chapter 11 Volume 3 P. 0006  Swedish special edition: Chapter 11 Volume 3 P. 0006  Spanish special edition: Chapter 12 Volume 2 P. 0092  Portuguese special edition Chapter 12 Volume 2 P. 0092 COOPERATION AGREEMENT BETWEEN THE EUROPEAN ATOMIC ENERGY COMMUNITY AND THE INTERNATIONAL ATOMIC


In [43]:
from pprint import pprint
from lmqg import TransformersQG
import pandas as pd

# Initialize the model
model = TransformersQG(model='lmqg/t5-base-squad-qg')

# Read context and answer data from CSV file
df = pd.read_csv("/chunks.csv")  # Adjust the file path as needed
contexts = df['text'].tolist()[:10]  # Assuming the column containing context is named 'text'
  # Assuming the column containing answer is named 'answer'

# Model prediction
questions = model.generate_q(list_context=contexts)
pprint(questions)
# answer = model.generate_a(context)
# pprint(answer)


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2850: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io

['What is the name of the Cooperation Agreement between the European Atomic '
 'Energy Community and the International Atomic Energy Agency?',
 'What does the European Atomic Energy Community do?',
 'What is the purpose of Article II Representation 1?',
 'What shall the Agency and the Community do to ensure reciprocal '
 'representation at meetings convened under their respective auspices?',
 'What is the nature of the confidential nature of certain information and '
 'documents?',
 'What is the purpose of Article VI Notification to the United Nations and '
 'filing and recording?',
 'What language is used in Article IX of the Agreement?',
 'What did the European Atomic Energy Community prepare official translations '
 'of this Agreement into?',
 'What is the name of the agreement between the EUROPEAN ATOMIC ENERGY '
 'COMMUNITY and the INTERNATIONAL ATOMIC ENERGY AGENCY?',
 'What is the purpose of Article II Representation 1?']


In [36]:
rag_pipeline(questions[1])

Keyword arguments {'add_special_tokens': False} not recognized.




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

a supplementary research programme to be implemented by the joint research centre for the European Atomic Energy Community (1992-1995) (92/275/EEC)THE COUNCIL OF THE EUROPEAN COMMUNITIES,  Having regard to the Treaty establishing the European Atomic Energy Community, and in particular Article 7 thereof,  Having regard to the proposal from the Commission submitted after consultation of the Scientific and Technical Committee(1) ,  Having regard to the opinion of the European Parliament(2) ,  Having regard to the opinion of the Economic and Social Committee(3) ,

the European Atomic Energy Community represented by the Commission of the European Communities and the Department of Energy o

{'query': 'What does the European Atomic Energy Community do?',
 'result': ' Based on the text provided, it appears that the European Atomic Energy Community (Euratom) is involved in research and development related to fusion energy. Specifically, the text mentions a supplementary research program to be implemented by the joint research center for Euratom (1992-1995) and an agreement for cooperation between Euratom and the Department of Energy of the United States of America in the field of fusion energy research and development. Additionally, the text notes that all official languages of the European Union are equally valid, suggesting that the community operates in multiple languages.'}

In [37]:
rag_pipeline(questions[2])

Keyword arguments {'add_special_tokens': False} not recognized.




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Transposition
   Member States shall bring into force the laws, regulations and administrative provisions necessary to comply with Articles 1 to 9, 11, 14, 15 and 20 and Annexes I to V, VII and VIII by 20 November 2010. They shall forthwith communicate to the Commission the text of those provisions.
When Member States adopt these measures, they shall contain a reference to this Directive or shall be accompanied by such reference on the occasion of their official publication. They shall also include a statement that references in existing laws, regulations and administrative provisions to the Directive repealed by this Directive shall be construed as references to this Directive. Memb

{'query': 'What is the purpose of Article II Representation 1?',
 'result': ' Based on the provided text, it appears that Article II Representation 1 is related to the approval of an agreement between the European Community and the Republic of Poland regarding the modernization of the Polish oil sector. Specifically, the article seems to be stating that the European Community has approved the agreement on behalf of its member states. However, without more context or information about the specific agreement being referenced, it is difficult to provide a more detailed answer.'}

In [38]:
rag_pipeline(questions[3])

Keyword arguments {'add_special_tokens': False} not recognized.




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

may participate, without the right to vote, in the deliberations of that body and, where appropriate, of its committees with respect to items on their agenda in which the Community has an interest. 2. The Agency and the Community shall also make any necessary arrangements for ensuring reciprocal representation at appropriate meetings convened under their respective auspices. Article III Exchange of information and documents The Agency and the Community shall undertake a full exchange of information and documents, subject to such restrictions and arrangements as may be considered necessary by either Contracting Party to

with respect to items on their agenda in which the Community has

{'query': 'What shall the Agency and the Community do to ensure reciprocal representation at meetings convened under their respective auspices?',
 'result': ' Based on the text, the Agency and the Community shall make any necessary arrangements for ensuring reciprocal representation at appropriate meetings convened under their respective auspices.'}

In [48]:
from langchain.chains import ConversationalRetrievalChain
chat_history = []

qa_conversation = ConversationalRetrievalChain.from_llm(llm=llm,
    verbose=True,
    retriever=vectorstore.as_retriever(search_kwargs={"k":5}),
)
result = qa_conversation({"question": questions[1], "chat_history": chat_history})

Keyword arguments {'add_special_tokens': False} not recognized.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

a supplementary research programme to be implemented by the joint research centre for the European Atomic Energy Community (1992-1995) (92/275/EEC)THE COUNCIL OF THE EUROPEAN COMMUNITIES,  Having regard to the Treaty establishing the European Atomic Energy Community, and in particular Article 7 thereof,  Having regard to the proposal from the Commission submitted after consultation of the Scientific and Technical Committee(1) ,  Having regard to the opinion of the European Parliament(2) ,  Having regard to the opinion of the Economic and Social Committee(3) ,

the European Atomic Energy Community represented by the Commission of the European Communities and the Department of Energy of the United States of America in the 

In [49]:
result['answer']

' Based on the text provided, it appears that the European Atomic Energy Community (Euratom) is involved in research and development related to fusion energy. Specifically, the text mentions a supplementary research program to be implemented by the joint research center for Euratom (1992-1995) and an agreement for cooperation between Euratom and the Department of Energy of the United States of America in the field of fusion energy research and development. Additionally, the text notes that all official languages of the European Union are equally valid, suggesting that the community operates in multiple languages.'

In [61]:
import evaluate
import numpy as np
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")
bleu = bleu.compute(predictions=result["answer"], references=result["answer"])
rouge_score = rouge.compute(predictions=result["answer"],references=result["answer"])
print(rouge_score)
print(bleu)

{'rouge1': 0.8303715670436187, 'rouge2': 0.0, 'rougeL': 0.8303715670436187, 'rougeLsum': 0.8303715670436187}
{'bleu': 0.0, 'precisions': [1.0, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 526, 'reference_length': 526}
